In [243]:
import pandas as pd
import requests
import datetime # https://docs.python.org/3/library/datetime.html

In [244]:
link = 'https://data.brasil.io/dataset/covid19/caso.csv.gz'
response = requests.get(link) # acessando via "navegador"
response

<Response [200]>

In [245]:
data = datetime.datetime.now().strftime("%Y%m%d-%H%M")
file_name = f'download-{data}.csv.gz'
arquivo = open(file_name, 'wb') # response
arquivo.write(response.content)
arquivo.close()

In [246]:
df = pd.read_csv(file_name)

In [247]:
cidades = ['Três Corações', 'Santa Rita do Sapucaí', 'Cachoeira de Minas', 'Piranguinho', 'Itajubá', 'Poços de Caldas', 'Pouso Alegre']

In [248]:
df = df[df['city'].isin(cidades)]
df[df['city'].isin(cidades)]['city'].unique() # conferir se as cidades foram "pegas"

array(['Cachoeira de Minas', 'Itajubá', 'Piranguinho', 'Pouso Alegre',
       'Poços de Caldas', 'Santa Rita do Sapucaí', 'Três Corações'],
      dtype=object)

In [249]:
df = df.reset_index()[['date','city', 'confirmed', 'deaths']]

In [250]:
df = df.rename(columns={'date': 'data', 'city': 'cidade', 'confirmed': 'casos_confirmados', 'deaths': 'mortes_confirmadas'})

In [251]:
df.sort_values('data', inplace=True)

In [252]:
df.head()

,data,cidade,casos_confirmados,mortes_confirmadas
674,2020-03-20,Poços de Caldas,1,0
673,2020-03-21,Poços de Caldas,1,0
672,2020-03-23,Poços de Caldas,1,0
671,2020-03-24,Poços de Caldas,1,0
670,2020-03-25,Poços de Caldas,1,0


In [62]:
df_teste = df[df['cidade'] == 'Poços de Caldas'].copy()

In [66]:
df_teste['shift_casos'] = df_teste['casos_confirmados'].shift(fill_value=0)
df_teste.head()

,data,cidade,casos_confirmados,mortes_confirmadas,shift_casos
674,2020-03-20,Poços de Caldas,1,0,0
673,2020-03-21,Poços de Caldas,1,0,1
672,2020-03-23,Poços de Caldas,1,0,1
671,2020-03-24,Poços de Caldas,1,0,1
670,2020-03-25,Poços de Caldas,1,0,1


In [67]:
df_teste['casos_confirmados_dia'] = df_teste['casos_confirmados'] - df_teste['shift_casos']

In [69]:
df_teste['shift_mortes'] = df_teste['mortes_confirmadas'].shift(fill_value=0)
df_teste['mortes_confirmadas_dia'] = df_teste['mortes_confirmadas'] - df_teste['shift_mortes']

In [72]:
df_teste.head()

,data,cidade,casos_confirmados,mortes_confirmadas,shift_casos,casos_confirmados_dia,shift_mortes,mortes_confirmadas_dia
674,2020-03-20,Poços de Caldas,1,0,0,1,0,0
673,2020-03-21,Poços de Caldas,1,0,1,0,0,0
672,2020-03-23,Poços de Caldas,1,0,1,0,0,0
671,2020-03-24,Poços de Caldas,1,0,1,0,0,0
670,2020-03-25,Poços de Caldas,1,0,1,0,0,0


In [200]:
# df_resultado = pd.DataFrame([], columns=df.columns)
# df_resultado_2 = pd.DataFrame([], columns=df.columns)
for cidade in cidades:
    df_cidade = df[df['cidade'] == cidade].copy()
    
    df_cidade['shift_casos'] = df_cidade['casos_confirmados'].shift(fill_value=0)
    df_cidade['casos_confirmados_dia'] = df_cidade['casos_confirmados'] - df_cidade['shift_casos']
    
    df_cidade['shift_mortes'] = df_cidade['mortes_confirmadas'].shift(fill_value=0)
    df_cidade['mortes_confirmadas_dia'] = df_cidade['mortes_confirmadas'] - df_cidade['shift_mortes']
    # df['casos_confirmados_dia'] = df_cidade['casos_confirmados_dia']
    # df_resultado.append(df_cidade, sort=False)
    # df = pd.merge(df, df_cidade)
    # df_resultado_2 = pd.concat([df_resultado_2, df_cidade])
    

In [202]:
def processa_cidade(df, cidade):
    df_cidade = df[df['cidade'] == cidade].copy()
    
    df_cidade['shift_casos'] = df_cidade['casos_confirmados'].shift(fill_value=0)
    df_cidade['casos_confirmados_dia'] = df_cidade['casos_confirmados'] - df_cidade['shift_casos']
    
    df_cidade['shift_mortes'] = df_cidade['mortes_confirmadas'].shift(fill_value=0)
    df_cidade['mortes_confirmadas_dia'] = df_cidade['mortes_confirmadas'] - df_cidade['shift_mortes']
    
    return df_cidade

In [217]:
# resolvendo com o append
df_resultado = pd.DataFrame([], columns=df.columns)

for cidade in cidades:
    df_cidade = processa_cidade(df, cidade)
    df_resultado = df_resultado.append(df_cidade, sort=False)

df_resultado.head()

,data,cidade,casos_confirmados,mortes_confirmadas,shift_casos,casos_confirmados_dia,shift_mortes,mortes_confirmadas_dia
922,2020-04-28,Três Corações,1,0,0.0,1.0,0.0,0.0
921,2020-04-29,Três Corações,3,0,1.0,2.0,0.0,0.0
920,2020-04-30,Três Corações,3,0,3.0,0.0,0.0,0.0
919,2020-05-01,Três Corações,3,0,3.0,0.0,0.0,0.0
918,2020-05-02,Três Corações,4,0,3.0,1.0,0.0,0.0


In [218]:
#df_tc = processa_cidade(df, 'Três Corações')
#df_resultado_merge = pd.merge(df, df_tc, left_index=True, right_index=True)

#df_pocos = processa_cidade(df, 'Poços de Caldas')
#df_resultado_merge = pd.merge(df_resultado_merge, df_pocos, left_index=True, right_index=True)
#df_resultado_merge

In [253]:
# resolvendo com o concat
df_resultado_concat = pd.DataFrame(columns=df.columns)
df_pa = processa_cidade(df, 'Pouso Alegre')
df_resultado_concat = pd.concat([df_resultado_concat, df_pa])
df_tc = processa_cidade(df, 'Três Corações')
df_resultado_concat = pd.concat([df_resultado_concat, df_tc])
df_resultado_concat

/Users/lucheol/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until
/Users/lucheol/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


,casos_confirmados,casos_confirmados_dia,cidade,data,mortes_confirmadas,mortes_confirmadas_dia,shift_casos,shift_mortes
511,1,1.0,Pouso Alegre,2020-04-01,0,0.0,0.0,0.0
510,3,2.0,Pouso Alegre,2020-04-02,0,0.0,1.0,0.0
509,3,0.0,Pouso Alegre,2020-04-03,0,0.0,3.0,0.0
508,3,0.0,Pouso Alegre,2020-04-04,0,0.0,3.0,0.0
507,4,1.0,Pouso Alegre,2020-04-05,0,0.0,3.0,0.0
506,5,1.0,Pouso Alegre,2020-04-06,0,0.0,4.0,0.0
505,8,3.0,Pouso Alegre,2020-04-07,1,1.0,5.0,0.0
504,9,1.0,Pouso Alegre,2020-04-08,1,0.0,8.0,1.0
503,9,0.0,Pouso Alegre,2020-04-09,1,0.0,9.0,1.0
502,9,0.0,Pouso Alegre,2020-04-10,1,0.0,9.0,1.0


In [257]:
def processa_cidade(df, cidade):
    df_cidade = df[df['cidade'] == cidade].copy()
    
    df_cidade['shift_casos'] = df_cidade['casos_confirmados'].shift(fill_value=0)
    df_cidade['casos_confirmados_dia'] = df_cidade['casos_confirmados'] - df_cidade['shift_casos']
    
    df_cidade['shift_mortes'] = df_cidade['mortes_confirmadas'].shift(fill_value=0)
    df_cidade['mortes_confirmadas_dia'] = df_cidade['mortes_confirmadas'] - df_cidade['shift_mortes']
    
    return df_cidade[['casos_confirmados_dia', 'mortes_confirmadas_dia']]

In [262]:
# usando o update
df_result = df.copy() #não é necessário
df_result['casos_confirmados_dia'] = 0
df_result['mortes_confirmadas_dias'] = 0

for cidade in cidades:
    df_cidade = processa_cidade(df, cidade)
    df_result.update(df_cidade)
df_result

,data,cidade,casos_confirmados,mortes_confirmadas,casos_confirmados_dia,mortes_confirmadas_dias
674,2020-03-20,Poços de Caldas,1,0,1.0,0
673,2020-03-21,Poços de Caldas,1,0,0.0,0
672,2020-03-23,Poços de Caldas,1,0,0.0,0
671,2020-03-24,Poços de Caldas,1,0,0.0,0
670,2020-03-25,Poços de Caldas,1,0,0.0,0
669,2020-03-26,Poços de Caldas,1,0,0.0,0
668,2020-03-27,Poços de Caldas,1,0,0.0,0
667,2020-03-28,Poços de Caldas,1,0,0.0,0
666,2020-03-29,Poços de Caldas,2,0,1.0,0
665,2020-03-30,Poços de Caldas,2,0,0.0,0


In [264]:
df_final = df_result.rename(columns={'casos_confirmados': 'casos_confirmados_acumulado', 'mortes_confirmadas': 'mortes_confirmadas_acumulado'})

In [266]:
df_final.sort_values(['cidade', 'data'], inplace=True)

In [267]:
df_final

,data,cidade,casos_confirmados_acumulado,mortes_confirmadas_acumulado,casos_confirmados_dia,mortes_confirmadas_dias
107,2020-05-15,Cachoeira de Minas,1,0,1.0,0
106,2020-05-16,Cachoeira de Minas,1,0,0.0,0
105,2020-05-17,Cachoeira de Minas,1,0,0.0,0
104,2020-05-18,Cachoeira de Minas,1,0,0.0,0
103,2020-05-19,Cachoeira de Minas,1,0,0.0,0
102,2020-05-20,Cachoeira de Minas,1,0,0.0,0
101,2020-05-21,Cachoeira de Minas,1,0,0.0,0
100,2020-05-22,Cachoeira de Minas,1,0,0.0,0
99,2020-05-23,Cachoeira de Minas,1,0,0.0,0
98,2020-05-24,Cachoeira de Minas,1,0,0.0,0


Desafios
=====


1. Apagar o arquivo após o processamento
2. 